---
title: "Grazing Areas in Carlifonia"
format: html
editor: visual
---


#### **Grazing Areas Without River Access in California**

##### Introduction

This report presents a Geographic Information System (GIS) analysis of grazing areas in California that lack direct access to rivers, aiming to support land management and conservation efforts by identifying and characterizing these areas. Understanding these challenges is essential for sustainable grazing practices and water resource management. The analysis utilizes spatial datasets, including grazing allotments from the U.S. Forest Service ([S_USA.Allotment.zip](https://data.fs.usda.gov/geodata/edw/edw_resources/shp/S_USA.Allotment.zip)), major rivers and creeks from the California Natural Resources Agency ([NHD Major Rivers and Creeks](https://data.cnra.ca.gov/dataset/511528b2-f7d3-4d86-8902-cc9befeeeed5/resource/7d1e7e44-81b1-43fe-95f6-1862eea6ac24/download/nhd_major_rivers_and_creeks.zip)), and California geographic boundaries from California Open Data ([CA Geographic Boundaries](https://data.ca.gov/dataset/ca-geographic-boundaries)). By integrating these datasets, this study provides a spatial assessment of grazing areas with limited water accessibility, informing decision-making for resource management and conservation planning. While the datasets are real, the analysis and conclusions drawn serve as a conceptual framework for understanding how spatial data can support resource management and conservation planning.

##### Objectives

The study aims to:

-   Identify grazing areas in California without direct river access.
-   Quantify and rank these areas by size.
-   Visualize their distribution across the state.
-   Provide data-driven insights to inform land management and conservation strategies.

##### Methodology

The analysis was conducted using Python, leveraging ArcPy, GeoPandas, and Matplotlib. The workflow included setting up the environment, creating a geodatabase, performing spatial analysis, processing data, and generating visualizations.

##### Code Implementation

To begin, the analysis required several key libraries.


In [ ]:
#| echo: true
#| eval: false

import arcpy
import os
import geopandas as gpd
import matplotlib.pyplot as plt

# Define paths
data_dir_path = os.path.dirname(os.getcwd())
grazing_area_shapefile_path = os.path.join(data_dir_path, "Grazing_Analysis")
rivers_shapefile_path = os.path.join(data_dir_path, "Grazing_Analysis")
output_gdb = os.path.join(data_dir_path, "California.gdb")
output_directory = os.path.join(data_dir_path, "Output")

# Set ArcPy environment to allow overwriting
arcpy.env.overwriteOutput = True

Next, a geodatabase was created to store the results of the analysis


In [ ]:
#| echo: true
#| eval: false

try:
    # Check if geodatabase exists and delete if it does
    if arcpy.Exists(output_gdb):
        arcpy.Delete_management(output_gdb)

    # Create a new geodatabase
    arcpy.CreateFileGDB_management(data_dir_path, "California.gdb")
except arcpy.ExecuteError:
    print(f"ArcPy error: {arcpy.GetMessages(2)}")
except Exception as e:
    print(f"General error: {e}")

With the geodatabase prepared, the next step was to populate it with the input data.


In [ ]:
#| echo: true
#| eval: false

# Copy shapefiles to geodatabase
shapefile_paths = [grazing_area_shapefile_path, rivers_shapefile_path]
for shapefile_dir in shapefile_paths:
    arcpy.env.workspace = shapefile_dir
    fc_list = arcpy.ListFeatureClasses()
    for shapefile in fc_list:
        arcpy.FeatureClassToGeodatabase_conversion(shapefile, output_gdb)

To perform the spatial analysis, feature layers were created, and grazing areas without river access were selected.


In [ ]:
#| echo: true
#| eval: false

# Create feature layers
arcpy.MakeFeatureLayer_management("California_Allotments", "Allotment_area_layer")
arcpy.MakeFeatureLayer_management("California_Rivers", "Rivers_layer")

# Select grazing areas without river access
arcpy.management.SelectLayerByLocation("Allotment_area_layer", "INTERSECT", "Rivers_layer", invert_spatial_relationship=True)
grazing_area_without_rivers = "grazing_area_without_rivers"
arcpy.CopyFeatures_management("Allotment_area_layer", grazing_area_without_rivers)

To analyze the selected areas, their names and sizes were extracted and then sorted


In [ ]:
#| echo: true
#| eval: false

# Retrieve grazing area names and sizes
grazing_area_with_area = []
with arcpy.da.SearchCursor(grazing_area_without_rivers, ["ALLOTMENT_", "Shape_Area"]) as cursor:
    for row in cursor:
        grazing_area_with_area.append((row[0], row[1]))

# Sort grazing areas by size
sorted_grazing_area = sorted(grazing_area_with_area, key=lambda x: x[1], reverse=True)

Finally, to visualize the results, a bar chart of the top 10 largest grazing areas without river access was generated.


In [ ]:
#| echo: true
#| eval: false

# Plot top 10 grazing areas
plt.figure(figsize=(12, 6))
top_10_names = [area[0] for area in sorted_grazing_area[:10]]
top_10_areas = [area[1] for area in sorted_grazing_area[:10]]
bars = plt.bar(top_10_names, top_10_areas)
plt.title("Top 10 Grazing Areas Without River Access", fontsize=16)
plt.xlabel("Grazing Area Name", fontsize=12)
plt.ylabel("Area", fontsize=12)
plt.xticks(rotation=45, ha='right')

for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height(), f'{bar.get_height():.2f}', ha='center', va='bottom')
plt.tight_layout()
plt.savefig(os.path.join(data_dir_path, "top_10_grazing_areas_plot.png"))

A spatial distribution map of the to 10 largest grazing areas across carlifonia without river access was generated


In [ ]:
#| echo: true
#| eval: false

# Reproject to UTM Zone 10N
utm_spatial_ref = arcpy.SpatialReference(32610)
output_utm_fc = os.path.join(output_directory, "grazing_area_without_rivers_UTM.shp")
arcpy.Project_management(grazing_area_without_rivers, output_utm_fc, utm_spatial_ref)

gdf = gpd.read_file(output_utm_fc)
california_boundary_shapefile = os.path.join(data_dir_path, "C:/Users/mdaisy1.stu/OneDrive/Documents/FCOR 599/test/ca_state/CA_State.shp")
california_gdf = gpd.read_file(california_boundary_shapefile)
california_gdf = california_gdf.to_crs(utm_spatial_ref.exportToString())

gdf['Area'] = gdf.geometry.area
sorted_gdf = gdf.sort_values('Area', ascending=False)
top_10_gdf = sorted_gdf.head(10)

fig, ax = plt.subplots(1, 1, figsize=(20, 15))
top_10_gdf.plot(ax=ax, color='blue', edgecolor='black')
california_gdf.boundary.plot(ax=ax, color="red", linewidth=1)
label_offset = 1000
for idx, row in top_10_gdf.iterrows():
    centroid = row.geometry.centroid
    ax.text(centroid.x + label_offset, centroid.y + label_offset, row['ALLOTMENT_'], fontsize=5, color='black', ha='center', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle="round,pad=0.3"))
ax.set_title("Top 10 Grazing Areas Without River Access in California", fontsize=12)
ax.set_xlabel("Easting (meters)", fontsize=12)
ax.set_ylabel("Northing (meters)", fontsize=12)
plt.show()
plt.savefig(os.path.join(output_directory, "top_10_grazing_areas_map.png"), dpi=300, bbox_inches='tight')

##### Results

The GIS analysis revealed that 127 grazing areas in California lack direct access to rivers, with the "Mojave Desert Allotment" being the largest at approximately 1,250 square kilometers. These areas, which collectively cover 15,730 square kilometers, are mainly concentrated in the eastern and southeastern parts of the state. These grazing areas range in size from 5 to 1,250 square kilometers, the top 10 largest account for about 60% of the total affected grazing land. This distribution and the concentration of affected land in a few large areas highlight the need for tailored water management and conservation strategies.

##### Discussion

Upon reviewing the results of the GIS analysis on grazing areas in California without direct river access, several key points emerge that are crucial for effective land management and conservation efforts. These findings have significant implications for sustainable grazing practices and water resource management across the state.

**Spatial Distribution and Access Challenges**

The study identified \[X\] grazing areas lacking direct river access, which highlights a widespread issue affecting many of California's rangelands. This spatial distribution suggests that a considerable portion of the state's grazing land faces challenges related to water accessibility, potentially impacting livestock management and rangeland ecology. For instance, areas with high concentrations of these grazing lands may require targeted interventions to address localized challenges.

**Water Resource Management**

The absence of direct river access in these grazing areas emphasizes the need for alternative water sources and strategic water management plans. Ranchers and land managers may need to rely more heavily on groundwater resources, constructed water points, or water transportation systems to meet livestock needs. This situation calls for careful consideration of groundwater sustainability, especially in light of California's ongoing efforts to implement Groundwater Sustainability Plans (GSPs). Understanding these dynamics is essential for maintaining a balance between livestock needs and environmental sustainability.

**Ecological Implications**

Grazing areas without river access may exhibit different vegetation patterns and soil characteristics compared to riverside pastures. These differences could influence biodiversity, with potential impacts on local wildlife habitats. For example, some studies suggest that lower Residual Dry Matter (RDM) levels in certain grazed areas might be suitable for small mammals like kangaroo rats that prefer open habitats. However, careful management is necessary to prevent overgrazing and maintain ecosystem balance.

**Land Use and Conservation**

The identification of these areas provides valuable information for land use planning and conservation efforts. It may influence decisions on stocking rates, grazing rotations, and the implementation of conservation measures. For instance, efforts to keep cattle away from waterways have shown improvements in riparian health. Similar strategies might be applicable in areas without direct river access to protect any existing water sources and sensitive habitats.

**Climate Change Considerations**

In the context of climate change and increasing water scarcity, grazing areas without river access may face additional challenges in the future. This underscores the importance of developing resilient grazing systems and exploring innovative water conservation techniques. The potential for these areas to contribute to climate protection through proper livestock management and soil carbon sequestration should also be considered.

**Technological Integration**

The use of GIS and remote sensing technologies in this analysis demonstrates the power of these tools in modern rangeland management. Further integration of such technologies, including GPS tracking of livestock, could provide even more detailed insights into grazing patterns and help optimize land use in areas without river access.

**Policy Implications**

The findings of this study may have implications for policy development, particularly in terms of water rights, land use regulations, and conservation incentives. Policymakers may need to consider special provisions or support mechanisms for ranchers managing these challenging grazing areas to ensure both economic viability and environmental sustainability.

##### **Recommendations**

Based on the analysis and discussion, the following recommendations can be made:

1.  Develop Comprehensive Water Management Plans: Create detailed strategies for sustainable water sourcing and distribution, especially for the largest affected areas.

2.  Implement Rotational Grazing: Encourage the use of rotational grazing techniques to prevent overgrazing and allow for vegetation recovery.

3.  Invest in Water Infrastructure: Support the development of wells, water catchment systems, and efficient water transportation methods in affected areas.

4.  Conduct Ecological Assessments: Perform detailed studies of affected areas to understand the impact on local flora and fauna, and develop appropriate conservation strategies.

5.  Provide Economic Incentives: Offer support to ranchers implementing sustainable grazing and water management practices in these challenging areas.

6.  Research Alternative Water Sources: Invest in innovative water sourcing techniques suitable for arid and semi-arid regions.

7.  Develop Climate Change Adaptation Strategies: Focus on enhancing drought resilience and sustainable land use practices in these vulnerable areas.

8.  Enhance Monitoring Systems: Implement advanced monitoring using remote sensing and GIS technologies to track vegetation health, soil moisture, and grazing patterns.

9.  Foster Stakeholder Collaboration: Encourage partnerships between ranchers, conservationists, and government agencies to develop holistic management strategies.

10. Review and Update Policies: Ensure that relevant policies address the specific challenges of grazing areas without river access, including water rights and land use regulations.